# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
# import libraries
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)

remove_indx = np.array([])
# Removing messages that contain only spaces!
_,indeces = np.unique(df.message.values,return_inverse=True)
indeces_of_the_empty_messages = np.where((indeces==0)|(indeces==1)|(indeces==2))[0]
remove_indx = np.append(remove_indx,indeces_of_the_empty_messages)

# Removing nan catogries
for col_ind in range(4,40):
    remove_indx = np.append(remove_indx,df[df[df.columns[col_ind]].isna()].index.values)

df.drop(np.unique(remove_indx), inplace=True)

X = df.message.values
Y = df[df.columns[4:]].values
# print(df.columns[4:])
# print(np.unique(X))
# print(np.unique(Y))

In [58]:
remove_indx = np.array([])
for col_ind in range(4,40):
    remove_indx = np.append(remove_indx,df[df[df.columns[col_ind]].isna()].index.values)
    


In [64]:
remove_indx = np.array([])
# Removing messages that contain only spaces!
_,indeces = np.unique(df.message.values,return_inverse=True)
indeces_of_the_empty_messages = np.where((indeces==0)|(indeces==1)|(indeces==2))[0]
remove_indx = np.append(remove_indx,indeces_of_the_empty_messages)

# Removing nan catogries
for col_ind in range(4,40):
    remove_indx = np.append(remove_indx,df[df[df.columns[col_ind]].isna()].index.values)
print(remove_indx)

[ 12265.  12269.  12302. ...,  26342.  26343.  26344.]


array([12265, 12269, 12302])

In [59]:
chart.drop(np.unique(chart_drop_indeces), inplace=True)

array([ 26207.,  26208.,  26209., ...,  26342.,  26343.,  26344.])

In [3]:
for i in range(36):
    kl = Y[:,i]
    print(np.unique(kl))

[ 0.  1.  2.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]


In [40]:
uniq,indeces = np.unique(X,return_inverse=True)
print(indeces)
print(uniq[0])
# for ind,x in enumerate(uniq):
#     print(ind)

[22370 11024 11880 ..., 14736 15948  1338]
    


In [35]:
for ind,x in enumerate(np.unique(X)):
    if ind < 3:
        print("x = ",x)
        print(len(x))

x =      
4
x =                             
27
x =            .
11


In [4]:
import string
def tokenize(text):
    # find URLs
    detected_urls = re.findall(url_regex, text)
    # Replace the URLs with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Remove numbers from the text
    text = re.sub(r'\d+', '', text)
    
    # Remove panctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok.lower().strip())
        clean_tokens.append(clean_tok)

    return clean_tokens

In [36]:
for ind,message in enumerate(X):
    if ind == 12265: 
        tokens = tokenize(message)
        print("message = ",message)
        print("tokens = ",len(tokens), '\n')

message =      
tokens =  0 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
Y_pred = pipeline.predict(X_test)

ValueError: Unknown label type: (array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  1.,  0.],
       [ 1.,  1.,  0., ...,  0.,  0.,  1.]]), array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]]))

In [14]:
for i in range(36):
    print(classification_report(Y_test[:,i], Y_pred[:,i]))

             precision    recall  f1-score   support

        0.0       0.33      0.13      0.18      1536
        1.0       0.77      0.92      0.84      4965
        2.0       0.50      0.02      0.04        50

avg / total       0.66      0.73      0.68      6551

             precision    recall  f1-score   support

        0.0       0.83      0.98      0.90      5409
        1.0       0.43      0.07      0.12      1142

avg / total       0.76      0.82      0.76      6551

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6525
        1.0       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6551

             precision    recall  f1-score   support

        0.0       0.59      0.82      0.69      3826
        1.0       0.45      0.20      0.28      2725

avg / total       0.53      0.56      0.52      6551

             precision    recall  f1-score   support

        0.0       0.92      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
print(classification_report(Y_test[:,0], Y_pred[:,0])[precision])

NameError: name 'precision' is not defined

In [10]:
print(np.unique(Y_pred))

[ 0.  1.  2.]


In [ ]:
print(classification_report)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [50, 200],
        'clf__estimator__min_samples_split': [2, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.